# 构建数据集

## 导入包

In [1]:
import pickle
from utils import ZincPdbqt, generate_coor, write_data, ele_filter, read_dock_score
from tqdm import tqdm
import os
from functools import partial
import pandas as pd
os.getcwd()

'/mnt/e/Python_Project/SMTarRNA/project/utils'

## 从pkl文件读入对接能量分数

In [2]:
# origin_100K_data_path = 'data//100k'
# origin_100K_data_path = 'data/3a6p/100k'
origin_100K_data_path = '../data/4z4d/100k_apH'

dock_score_100k = read_dock_score(origin_100K_data_path)
# dock_score_10k = read_dock_score(origin_10K_data_path)
score_dict = dock_score_100k
# with open('E:/Python_Project/SMTarRNA/project/data/3a6p/3a6p_dock_origin_data/zinc_drug_like_3d_100k_dock_energy.pkl', 'rb') as f:
#     score_dict = pickle.load(f)

In [5]:
# 存入csv
dataframe = pd.Series(score_dict)
dataframe.to_csv('drug_like_100k_energy.csv')

In [7]:
energy_d = pd.read_csv('../data/drug_like_100k_energy.csv')
smiles_d = pd.read_csv('../data/zinc_600m_100k_smiles.csv')
smiles_d.set_index('zinc_id', inplace=True)
smiles_d['smiles_len'] = smiles_d['smiles'].apply(lambda x: len(x), 0)
smiles_d['smiles_len'] = smiles_d['smiles_len'].astype('int64')



In [32]:
smiles_d_cleaned = pd.Series()
for zinc_id in tqdm(energy_d['zinc_id']):
    # print(zinc_id)
    # print(smiles_d.loc[zinc_id]['smiles_len'].argmin())
    if type(smiles_d['smiles'].loc[zinc_id]) == str:
        smiles_d_cleaned[zinc_id] = smiles_d['smiles'].loc[zinc_id]
    else:
        smiles_d_cleaned[zinc_id] = smiles_d['smiles'].loc[zinc_id].iloc[smiles_d.loc[zinc_id]['smiles_len'].argmin()]

    # break

/tmp/ipykernel_30285/939222829.py:1: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  smiles_d_cleaned = pd.Series()
100%|██████████| 100444/100444 [11:04<00:00, 151.15it/s]


In [40]:
# energy_d.columns
smiles_d_cleaned

ZINC001484687115      CO[C@@H](CC(C)C)C(=O)N1CCO[C@@H](CNCc2nnnn2C)C1
ZINC000971930080     CCC[N@H+](C)[C@H]1CCN(C(=O)C[C@H]2CC(C)(C)CO2)C1
ZINC000982286002    COc1ccc(CN2C[C@]3(C)CN(C(=O)C4(C)CC4)C[C@]3(C)...
ZINC001482054465       C=C(C)CC[NH2+]C[C@H]1CCCN1C(=O)[C@H]1CCC(=O)N1
ZINC000944737603       Cc1coc(C(=O)NC[C@H]2CN(C(=O)c3csc(C)n3)CCO2)c1
                                          ...                        
ZINC001083284498    CC(C)=CCN1C[C@H]2OCCN(C(=O)c3cn(CC(N)=O)nn3)[C...
ZINC001364849572             COCc1nc(CNS(=O)(=O)[C@@H]2COC[C@H]2O)cs1
ZINC000491008935                CCOC(=O)N(C)C1CCN(CC(=O)N(C)C2CC2)CC1
ZINC000262257209           C[C@H](NC(=O)C[C@@H]1COCCN1)c1cc(Cl)ccc1Cl
energy              0        -6.981
1        -6.090
2        -7.73...
Length: 100445, dtype: object

In [44]:
# smiles_d_cleaned['energy'] = energy_d['energy']
# energy_d.set_index('zinc_id', inplace=True)
# out_dataframe = pd.DataFrame(data=dict(smiles=smiles_d_cleaned, energy=energy_d['energy']))
# out_dataframe = pd.merge(left=smiles_d_cleaned, right=energy_d['energy'], )
out_dataframe = pd.concat([smiles_d_cleaned, energy_d['energy']], axis=1)
# energy_d['smiles'] = smiles_d_cleaned
out_dataframe.to_csv('drug_like_100k_smiles_dataset.txt', index=False)
# smiles_d['smiles'].loc[zinc_id].iloc[smiles_d.loc[zinc_id]['smiles_len'].argmin()]

In [3]:
print(len(score_dict.keys()))
list(score_dict.keys())[0]

100444


'ZINC001484687115'

## 读入小分子的结构

In [4]:
ligands_file_name = 'zinc/zinc_drug_like_3d_100k_rand.pdbqt.gz'
elements_list = ['C', 'H', 'O', 'N', 'S', 'P', 'BR', 'CL', 'F', 'I']
transform = partial(ele_filter, elements_list=elements_list)
ligands = ZincPdbqt(ligands_file_name, filter_=[transform])

In [4]:
len(ligands.zinc_id)
with open('zinc_id.txt', 'w') as f:
    f.write('\n'.join(ligands.zinc_id))


## 构建数据集

In [5]:
# ligands.zinc_id
# print(len(ligands))
# print(ligands.zinc_id[0])
# score_dict[ligands.zinc_id[500]]
# elements_list = ['C', 'H', 'O', 'N', 'S', 'P', 'BR', 'CL', 'F', 'I']
f = open('4z4d_100K_rand.txt', 'w')
f.write('e(kcalmol^-1)\n')
data_dict = dict()
for ligands_id, ligands_model in tqdm(ligands, desc='prepare'):
    if ligands_id not in list(score_dict.keys()):
        print(f'here wrong {ligands_id}')
        continue
    data_dict['score'] = score_dict[ligands_id]
    data_dict['pos'] = generate_coor(ligands_model)
    data_dict['id'] = ligands_id
    f = write_data(f, data_dict)
    # break
f.close()

prepare: 100%|██████████| 100446/100446 [01:47<00:00, 934.51it/s]


In [7]:
score_dict = dict()
for i in range(102):
    ligands_file_name = f'/mnt/e/Python_Project/SMTarRNA/project/data/3a6p/3a6p_dock_origin_data/zinc_drug_like_3d_100k_dock_results_{i}.pdbqt.gz'
    ligands = ZincPdbqt(ligands_file_name)
    score_dict.update(ligands.scores)
    # break


In [8]:
len(score_dict.keys())

101442